# Deep Seeeion 8주차 과제

tweets를 기반으로 sentiment를 예측하는 task를 수행하고자 합니다. 

1. 데이터셋에 맞는 전처리를 진행해주세요.
2. 실습 파일을 참고하여 Sentiment_RNN, Sentiment_LSTM, Sentiment_GRU class를 각각 완성해주세요.
3. 3가지 모델에서의 성능을 비교해주세요.

In [48]:
import pandas as pd
import numpy as np
import random
import re

import torch
import torch.nn as nn

from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

### Reading Data

In [58]:
tweets = pd.read_csv('Tweets.csv')
tweets.head()

,Unnamed: 0,textID,text,selected_text,sentiment
0,0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,2,088c60f138,my boss is bullying me...,bullying me,negative
3,3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [50]:
#na값 제거 해줘야함
tweets.dropna(how='any',axis=0,inplace=True)

In [15]:
tweets.sentiment.unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [16]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(tweets['text'], tweets['sentiment'], stratify=tweets['sentiment'], random_state=220510, test_size=.2)

# twitter roberta tokenizer

In [47]:
dataset

Dataset({
    features: ['textID', 'text', 'selected_text', 'sentiment'],
    num_rows: 27481
})

In [63]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric

#MODEL = "klue/roberta-base"
INPUT = "Tweets.csv"
MAX_LEN = 256
dataset = load_dataset("csv", data_files=INPUT,split='train')
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
def example_fn(examples):
    outputs = tokenizer(examples['text'],padding=True, max_length=MAX_LEN,truncation=True)
    if 'sentiment' in examples:
        outputs["sentiment"] = examples["sentiment"]
    return outputs

dataset = dataset.map(example_fn, remove_columns=['text', 'sentiment'])

dataset = dataset.train_test_split(0.2)


Using custom data configuration default-6085decd612914ef
Reusing dataset csv (C:\Users\Quantec\.cache\huggingface\datasets\csv\default-6085decd612914ef\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/27480 [00:00<?, ?ex/s]

In [64]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'textID', 'selected_text', 'sentiment', 'input_ids', 'attention_mask'],
        num_rows: 21984
    })
    test: Dataset({
        features: ['Unnamed: 0', 'textID', 'selected_text', 'sentiment', 'input_ids', 'attention_mask'],
        num_rows: 5496
    })
})

In [117]:
X_train=pd.DataFrame(dataset['train'])['input_ids']
X_test=pd.DataFrame(dataset['test'])['input_ids']
y_train=pd.DataFrame(dataset['train'])['sentiment']
y_test=pd.DataFrame(dataset['test'])['sentiment']

### Preprocessing

In [17]:
def preprocessing(data):
    data = data.map(lambda x: x.lower())
    return data

### Tokenization

In [18]:
# 띄어쓰기 기준 tokenization
def sep_based_tok(sentences):
    toks = sentences.map(lambda x: x.split())
    return toks

### Word Embedding

In [19]:
# label encoding
def le_emb(toks, token_cb):
    le = {token:i for i, token in enumerate(token_cb)}
    embs = toks.map(lambda x: [le.get(tok, 0) for tok in x])
    return embs

In [20]:
def sentence_vectorization(sentences, tok_method, emb_method):
    sentences = preprocessing(sentences) # preprocessing
    toks = tok_method(sentences) # tokenization
    token_cb = ['<unk>'] + list({word for sentence in toks for word in sentence}) # make vocabulary
    embs = emb_method(toks, token_cb) # word embedding
    return embs

In [21]:
X_train = sentence_vectorization(X_train, sep_based_tok, le_emb)
X_test = sentence_vectorization(X_test, sep_based_tok, le_emb)

### Modeling

In [118]:
# 장비 할당
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [122]:
# 패딩: 입력 벡터 길이 맞추기
X_train = torch.FloatTensor(pad_sequences(X_train)).unsqueeze(2)
X_test = torch.FloatTensor(pad_sequences(X_test)).unsqueeze(2)

In [119]:
# 타겟 데이터 전처리
idx2label = dict(enumerate(y_train.unique()))
label2idx = {label:idx for idx, label in idx2label.items()}
y_train = [label2idx[x] for x in y_train]
y_test = [label2idx[x] for x in y_test]
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [127]:
from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataset, val_dataset = train_test_split(train_dataset, random_state=220510, test_size=.2)

In [128]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
val_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

## RNN

In [129]:

class Sentiment_RNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=3, dropout=.3, bidirectional=True):
        super(Sentiment_RNN, self).__init__()
        self.rnn = nn.RNN(input_size=input_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            dropout = dropout,
                            bidirectional=bidirectional)
        if bidirectional:
            self.fc = nn.Linear(hidden_size*2, 4)
        else:
            self.fc = nn.Linear(hidden_size, 4)
    def forward(self, x):
        output, hidden = self.rnn(x)
        last_output = output[:,-1,:]
        return self.fc(last_output)

## LSTM

In [133]:

class Sentiment_LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=3, dropout=.3, bidirectional=True):
        super(Sentiment_LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            dropout = dropout,
                            bidirectional=bidirectional)
        if bidirectional:
            self.fc = nn.Linear(hidden_size*2, 4)
        else:
            self.fc = nn.Linear(hidden_size, 4)
    def forward(self, x):
        output, hidden = self.lstm(x)
        last_output = output[:,-1,:]
        return self.fc(last_output)

## GRU

In [111]:

class Sentiment_GRU(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=3, dropout=.3, bidirectional=True):
        super(Sentiment_GRU, self).__init__()
        self.gru = nn.GRU(input_size=input_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            dropout = dropout,
                            bidirectional=bidirectional)
        if bidirectional:
            self.fc = nn.Linear(hidden_size*2, 4)
        else:
            self.fc = nn.Linear(hidden_size, 4)
    def forward(self, x):
        output, hidden = self.gru(x)
        last_output = output[:,-1,:]
        return self.fc(last_output)

In [130]:
def train(model, train_dataloader, optim, loss_fc):
    model.train()
    train_loss = 0
    correct = 0
    num_samples = 0
    for idx, batch in enumerate(train_dataloader):
        text = batch[0].to(device)
        target = batch[1].to(device)
        
        optimizer.zero_grad()
        preds = model(text)
        loss = loss_fc(preds, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        prediction = preds.max(1, keepdim=True)[1]
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        
        num_samples += target.size(0)
        
    train_loss /= num_samples
    train_acc = 100 * correct /num_samples
    
    print(f'Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')

def evaluate(model, test_dataloader, loss_fc):
    model.eval() # 모델을 평가상태로 지정
    test_loss = 0
    correct = 0
    num_samples = 0
    with torch.no_grad(): # 평가 과정에서 gradient 업데이트를 하지 않기 위해
        for batch in test_dataloader:
            text = batch[0].to(device)
            target = batch[1].to(device)
            output = model(text)
            test_loss += criterion(output, target).item()
            
            prediction = output.max(1, keepdim=True)[1] # 벡터 값 내 최대값으로 예측
            correct += prediction.eq(target.view_as(prediction)).sum().item()
            
            num_samples += target.size(0)
            
    test_loss /= num_samples
    test_accuracy = 100 * correct /num_samples
    
    return test_loss, test_accuracy

In [131]:
def train_epochs(model, train_dataloader, val_dataloader, optim, loss_fc, n_epoch):
    for epoch in range(n_epoch):
        print(f'-----Epoch : {epoch+1}-----')
        train(model, train_dataloader, optim, loss_fc)
        valid_loss, valid_acc = evaluate(model, val_dataloader, loss_fc)
        print(f'Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

In [134]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
random.seed(0)

# 모델을 변경해서 사용해주세요.
model = Sentiment_LSTM().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

train_epochs(model, train_dataloader, val_dataloader, optimizer, criterion, 10)

-----Epoch : 1-----
Train Loss : 0.04282 | Train Acc : 31.32
Valid Loss : 0.04194 | Valid Acc : 32.18
-----Epoch : 2-----
Train Loss : 0.04009 | Train Acc : 38.47
Valid Loss : 0.03805 | Valid Acc : 39.37
-----Epoch : 3-----
Train Loss : 0.03646 | Train Acc : 40.21
Valid Loss : 0.03568 | Valid Acc : 39.37
-----Epoch : 4-----
Train Loss : 0.03518 | Train Acc : 40.22
Valid Loss : 0.03503 | Valid Acc : 39.37
-----Epoch : 5-----
Train Loss : 0.03475 | Train Acc : 40.33
Valid Loss : 0.03476 | Valid Acc : 39.37
-----Epoch : 6-----
Train Loss : 0.03453 | Train Acc : 40.37
Valid Loss : 0.03461 | Valid Acc : 39.37
-----Epoch : 7-----
Train Loss : 0.03441 | Train Acc : 40.29
Valid Loss : 0.03452 | Valid Acc : 39.46
-----Epoch : 8-----
Train Loss : 0.03433 | Train Acc : 40.39
Valid Loss : 0.03445 | Valid Acc : 39.46
-----Epoch : 9-----
Train Loss : 0.03426 | Train Acc : 40.37
Valid Loss : 0.0344 | Valid Acc : 39.46
-----Epoch : 10-----
Train Loss : 0.03422 | Train Acc : 40.39
Valid Loss : 0.03437 

### Prediction

In [75]:
#GRU + twitter tokenizer
test_loss, test_acc = evaluate(model, test_dataloader, criterion)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.03424 | Test Acc : 41.3


In [135]:
#LSTM + tiwtter tokenizer
test_loss, test_acc = evaluate(model, test_dataloader, criterion)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.03425 | Test Acc : 41.3


In [115]:
#RNN + twitter tokenizer
test_loss, test_acc = evaluate(model, test_dataloader, criterion)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.03418 | Test Acc : 41.3
